In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from collections import Counter
import warnings
import pickle
from catboost import CatBoostClassifier
warnings.filterwarnings("ignore")

In [ ]:
import numba
from numba import jit

In [ ]:

def model_fit1(X_train, y_train):
    categorical_features_indices = [ 0,  2,  3,  6, 15, 16, 19, 28, 29, 32, 41, 42, 45, 54, 55, 58, 67, 68, 71, 72, 73, 74, 75, 76, 78, 79]
    model = CatBoostClassifier(iterations=300, depth=5,cat_features=categorical_features_indices,learning_rate=0.5, loss_function='Logloss',
                            logging_level='Verbose')
    model.fit(X_train,y_train,eval_set=(X_train, y_train))
    return model

### time

In [ ]:
train = pd.read_csv('train/is_train_20190701.txt')
# train = pd.merge(train[['link','current_slice_id','future_slice_id']][(train.future_slice_id <= 50)],train,how='left') 


In [ ]:
train1 = pd.read_csv('train/is_train_201907'+str(16)+'.txt')
train = train.append(train1)

In [ ]:
pro = pd.read_csv('per_road_pro.txt')

In [ ]:
# train = pd.read_csv('is_train_data/is_train_20190723.txt')
train = pd.merge(train, pro, how='left')

In [ ]:
train = train.drop('pro_0',1)
train = train.drop('pro_1',1)
train = train.drop('pro_2',1)

In [ ]:
y = train.label
train = train.drop('label',1)

### 过采样

In [ ]:
from imblearn.over_sampling import RandomOverSampler
 
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_sample(train, y)

### 下采样

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
 
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_sample(train, y)

In [ ]:
X_resampled

In [ ]:
train = train.drop('current_speed_min',1)
train = train.drop('current_eta_min',1)
train = train.drop('current_cnt_min',1)
train = train.drop('current_state_min',1)

train = train.drop('his_28_speed_min',1)
train = train.drop('his_28_eta_min',1)
train = train.drop('his_28_cnt_min',1)
train = train.drop('his_28_state_min',1)

train = train.drop('his_21_speed_min',1)
train = train.drop('his_21_eta_min',1)
train = train.drop('his_21_cnt_min',1)
train = train.drop('his_21_state_min',1)

train = train.drop('his_14_speed_min',1)
train = train.drop('his_14_eta_min',1)
train = train.drop('his_14_cnt_min',1)
train = train.drop('his_14_state_min',1)

train = train.drop('his_7_speed_min',1)
train = train.drop('his_7_eta_min',1)
train = train.drop('his_7_cnt_min',1)
train = train.drop('his_7_state_min',1)

In [ ]:
s = train.columns.values

In [ ]:
train_1 = pd.read_csv('train//is_train_20190730.txt')
y_5 = train_1.label
train_1 = pd.merge(train_1, pro, how='left')
train_1 = train_1.drop('label',1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(train,y,test_size=0.2 , random_state=1234)
classweight = {0:0.4,1:0.4,2:0.2}

model = CatBoostClassifier(iterations=200,depth=6,l2_leaf_reg=8,learning_rate=0.5,random_strength=1,
                                      loss_function='MultiClassOneVsAll',logging_level='Verbose')

model.fit(X_train,y_train,eval_set=(X_validation,y_validation))

In [ ]:
from sklearn.model_selection import train_test_split
# X_train, X_validation, y_train, y_validation = train_test_split(train.loc[:,'time_diff':'width'],y,test_size=0.25 , random_state=1234)
X_train, X_validation, y_train, y_validation = train_test_split(train,y,test_size=0.2 , random_state=1234)

In [ ]:
categorical_features_indices = np.where(X_train.dtypes != np.float)[0]

In [ ]:
classweight = {0:0.4,1:0.4,2:0.2}
model = CatBoostClassifier(iterations=200,depth=6,l2_leaf_reg=8,learning_rate=0.5,random_strength=1,
                                      loss_function='MultiClassOneVsAll',logging_level='Verbose')

model.fit(X_train,y_train,eval_set=(train_1,y_5),verbose=False)

In [ ]:
#w = np.array((y_train+1))
# model.fit(X_train,y_train,eval_set=(X_validation,y_validation),sample_weight==w)
model.fit(X_train,y_train,eval_set=(X_validation,y_validation))

In [ ]:
 print(classification_report(y_validation, model.predict(X_validation), digits=4))

In [ ]:
 print(classification_report(y_validation, model.predict(X_validation), digits=4))

In [ ]:
import matplotlib.pyplot as plt 
fea_ = model.feature_importances_
fea_name = model.feature_names_
plt.figure(figsize=(10, 50))
plt.barh(fea_name,fea_,height =0.5)

### 测试集1

In [ ]:
train_1 = pd.read_csv('train//is_train_20190730.txt')
y_5 = train_1.label
train_1 = pd.merge(train_1, pro, how='left')
train_1 = train_1.drop('label',1)

### 测试集2

In [ ]:
train_1 = pd.read_csv('is_train_data/is_train_20190701.txt')
train_1 = pd.merge(train_1[['link','current_slice_id','future_slice_id']][(train_1.future_slice_id <= 50)],train_1,how='left') 
for i in tqdm(range(25, 31)):
    train1 = pd.read_csv('is_train_data/is_train_201907'+str(i)+'.txt')
    train_1 = train_1.append(pd.merge(train1[['link','current_slice_id','future_slice_id']][(train1.future_slice_id <= 50)],train1,how='left'))

train_1 = pd.merge(train_1, pro, how='left')
y_5 = train_1.label
train_1 = train_1.drop('label',1)
train_1 = train_1.drop('is_high',1)

In [ ]:
train_1 = train_1.drop('pro_0',1)
train_1 = train_1.drop('pro_1',1)
train_1 = train_1.drop('pro_2',1)

In [ ]:

train_1 = train_1.drop('current_speed_min',1)
train_1 = train_1.drop('current_eta_min',1)
train_1 = train_1.drop('current_cnt_min',1)
train_1 = train_1.drop('current_state_min',1)

train_1 = train_1.drop('his_28_speed_min',1)
train_1 = train_1.drop('his_28_eta_min',1)
train_1 = train_1.drop('his_28_cnt_min',1)
train_1 = train_1.drop('his_28_state_min',1)

train_1 = train_1.drop('his_21_speed_min',1)
train_1 = train_1.drop('his_21_eta_min',1)
train_1 = train_1.drop('his_21_cnt_min',1)
train_1 = train_1.drop('his_21_state_min',1)

train_1 = train_1.drop('his_14_speed_min',1)
train_1 = train_1.drop('his_14_eta_min',1)
train_1 = train_1.drop('his_14_cnt_min',1)
train_1 = train_1.drop('his_14_state_min',1)

train_1 = train_1.drop('his_7_speed_min',1)
train_1 = train_1.drop('his_7_eta_min',1)
train_1 = train_1.drop('his_7_cnt_min',1)
train_1 = train_1.drop('his_7_state_min',1)


In [ ]:
kkk = model.predict(train_1)
print(classification_report(y_5, kkk, digits=4))
report = f1_score(y_5, kkk, average=None)
print('Score: ', report[0] * 0.2 + report[1] * 0.2 + report[2] * 0.6)

In [ ]:
kkk = model.predict(train_1)
print(classification_report(y_5, kkk, digits=4))
report = f1_score(y_5, kkk, average=None)
print('Score: ', report[0] * 0.2 + report[1] * 0.2 + report[2] * 0.6)

### 无high

In [ ]:
kkk = model.predict(train_1)
print(classification_report(y_5, kkk, digits=4))
report = f1_score(y_5, kkk, average=None)
print('Score: ', report[0] * 0.2 + report[1] * 0.2 + report[2] * 0.6)

### 有high

In [ ]:
print(classification_report(y_5, model.predict(train_1), digits=4))
report = f1_score(y_5, model.predict(train_1), average=None)
print('Score: ', report[0] * 0.2 + report[1] * 0.2 + report[2] * 0.6)

### 16号预测

In [ ]:
 print(classification_report(y_validation, model.predict(X_validation), digits=4))

In [ ]:
num = []
for i in range(504891):
    s = k['linkid_' 'label_' 'current_slice_id_' 'future_slice_id'][i]
    n = len(s)
    k2 = 0
    num2 = []
    cnt = 0
    for j in range(n):
        if s[j] == ' ':
            k2 += 1
            if k2 != 2:
                num2.append(cnt)
            cnt = 0
            continue
        if s[j] == '-': continue
        cnt = cnt * 10 + int(s[j])
        if j == n - 1:
            num2.append(cnt)
    num2.append(3)
    num.append(num2)

In [ ]:
num[504890]

In [ ]:
import csv
f = open(f'f.csv','w',encoding='utf-8',newline='')
csv_writer = csv.writer(f)
csv_writer.writerow(["link","current_slice_id","future_slice_id","label"])
csv_writer.writerows(num)
f.close()

In [ ]:
import pycuda.driver as drv